In [1]:
import os 
import json
from ai2thor.controller import Controller
import time
import pickle
import copy

In [2]:
def run(floorPlan, object_poses, actions, init_action, object_toggles, dirty_and_empty , turk_annotations, controller, task_type):
    
    controller.reset(scene_name =  floorPlan)
    if len(object_toggles) > 0:
            controller.step((dict(action='SetObjectToggles', objectToggles=object_toggles)))

    if dirty_and_empty:
            controller.step(dict(action='SetStateOfAllObjects',
                               StateChange="CanBeDirty",
                               forceAction=True))
            controller.step(dict(action='SetStateOfAllObjects',
                               StateChange="CanBeFilled",
                               forceAction=False))

    tmp = dict( action ='SetObjectPoses', objectPoses=object_poses)
 
    controller.step(tmp)
   
    # frame  = controller.last_event.frame
    controller.step(init_action)
    items = []
    subgoals = 0
    for action in actions:
        subgoals = max(action['high_idx'], subgoals)
    subgoals += 1
    for action in actions:
        precondition = controller.last_event.frame
        # print(action['api_action'])
        controller.step(action['api_action'] )
        # print(controller.last_event.metadata['lastActionSuccess'])
        posteffect = controller.last_event.frame
        item = {
            'precondition': precondition, 
            'posteffect': posteffect, 
            'action' : action['api_action'], 
            'subgoals': subgoals, 
            'current_goal': action['high_idx'], 
            'currentprogress': float(action['high_idx'])/float(subgoals), 
            'low_descs': [ a['high_descs'][action['high_idx']] for a in  turk_annotations] , 
            'task_descs': [ a['task_desc'] for a in turk_annotations ],
            'task_type': task_type
        }
        items.append(item)
        # cur_high_idx = 0
        
        # for a in turk_annotations:
        #     try:
        #         item['low_descs'] = None
        #         item['task_desc'] = None
        #         item['low_descs'] = a['high_descs'][action['high_idx']]
        #         item['task_desc'] = a['task_desc']
               
        #         if item['low_descs'] and item['task_desc']:
        #             items.append(copy.deepcopy(item))
                
        #     except:
        #         pass

    return items



In [3]:
split = 'train'
tasks = os.listdir('data/full_2.1.0/' + split)
cnt = 0
controller = Controller()
controller.start(player_screen_height=300,
                 player_screen_width=300)
from tqdm import tqdm
table = []
for task in tqdm(tasks):
    task_path =  os.path.join('data/full_2.1.0/' + split, task)
    trials = os.listdir(task_path)
    
    for trial in trials:
        path = os.path.join('data/full_2.1.0/' + split, task, trial, 'traj_data.json')
        
        with open(path) as f:
            data = json.load(f)

        floorPlan = data['scene']['floor_plan']
        object_poses = data['scene']['object_poses']
        actions = data['plan']['low_actions']
        init_action = data['scene']['init_action']
        object_toggles  = data['scene']['object_toggles']
        dirty_and_empty = data['scene']['dirty_and_empty']
        turk_annotations =  data['turk_annotations']['anns']
        try:
            eposide = run(floorPlan, object_poses, actions, init_action, object_toggles, dirty_and_empty, turk_annotations , controller, data['task_type'])
            table.append(eposide)
        except:
            pass
        
    
    if len(table) > 500 or task == tasks[-1]:
        with open('dataset_'+split+'_'+str(cnt)+'.pickle', 'wb') as handle:
            pickle.dump(table, handle, protocol=pickle.HIGHEST_PROTOCOL)
        cnt += 1
        table = []
    
        # for action in actions:
        #         print(action['api_action'])
       
controller.stop()

100%|██████████| 2435/2435 [2:55:28<00:00,  4.32s/it]


In [4]:
data.keys()

dict_keys(['images', 'pddl_params', 'plan', 'scene', 'task_id', 'task_type', 'turk_annotations'])

In [5]:
data['task_type']

'pick_and_place_simple'

In [6]:
data['turk_annotations']

{'anns': [{'assignment_id': 'ARL7HOWLEHNOP_3NG53N1RLY0AP91Z0UYSR74E2JL8P4',
   'high_descs': ['Turn across the room, turn right, and walk to the refrigerator.',
    'Open the refrigerator, take the glass, and shut the refrigerator door.',
    'Turn around and walk to the cabinet.',
    'Place the glass in the second cabinet door on the right.'],
   'task_desc': 'Place the chilled glass in the cabinet.',
   'votes': [1, 1, 1]},
  {'assignment_id': 'A1ELPYAFO7MANS_3ZR9AIQJUEQ6IRAD76TPVUN1SQM40X',
   'high_descs': ['Walk straight and hang right to face the fridge.',
    'Take the glass from the top shelf of the fridge.',
    'Turn around and take a step forward.',
    'Place the glass in the bottom right cabinet.'],
   'task_desc': 'Place a chilled glass in a cabinet.',
   'votes': [1, 1, 1]},
  {'assignment_id': 'AJQGWGESKQT4Y_3AUQQEL7U8A5C4TSR2UC4KNF85HV0T',
   'high_descs': ['Go forward and then turn right to face the fridge.',
    'Open the fridge and take the vase out and then shut t